In [2]:
from pyspark.sql import functions as F

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 4, Finished, Available, Finished)

In [3]:
df_contratos = spark.sql("SELECT * FROM lakehouseSilver.contrato")

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 5, Finished, Available, Finished)

In [4]:
dim_clientes = spark.sql("""
    SELECT skCliente, CAST(codigoCliente AS STRING) AS codigoCliente
    FROM lakehouseSilver.dimensionClienteSilver
""")

dim_activos = spark.sql("""
    SELECT skActivo, CAST(numContrato AS STRING) AS numContrato
    FROM lakehouseSilver.dimensionActivosSilver
""")

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 6, Finished, Available, Finished)

In [5]:
df_contratos = (
    df_contratos
    .withColumn("numContrato", F.trim(F.col("numContrato").cast("string")))
    .withColumn("idCliente", F.trim(F.col("idCliente").cast("string")))
)

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 7, Finished, Available, Finished)

In [6]:
dim_clientes = dim_clientes.withColumn("codigoCliente", F.trim(F.col("codigoCliente")))
dim_activos = dim_activos.withColumn("numContrato", F.trim(F.col("numContrato")))

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 8, Finished, Available, Finished)

In [7]:
df_fact = (
    df_contratos
    .join(dim_clientes, df_contratos.idCliente == dim_clientes.codigoCliente, "left")
    .join(dim_activos, on="numContrato", how="left")
)

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 9, Finished, Available, Finished)

In [8]:
df_fact = (
    df_fact
    .withColumn("FechaCreacion", F.current_timestamp())
    .withColumn("EsActual", F.lit(True))
)

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 10, Finished, Available, Finished)

In [10]:
print(df_fact.columns)

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 12, Finished, Available, Finished)

['numContrato', 'idContrato', 'idCliente', 'fechaInicio', 'montoActivos', 'montoSaldoAnterior', 'montoNuevaCompra', 'montoAbono', 'moneda', 'montoLeasing', 'plazoOriginal', 'plazo', 'tasa', 'tipoArrendamiento', 'montoDeposito', 'montoDepositoAbonado', 'tipoCuota', 'montoSinSeguro', 'montoSeguroContrato', 'montoMantenimiento', 'montoSeguroMantenimiento', 'montoReserva', 'montoCompraFinal', 'montoProveedor', 'diaPago', 'fechaPrimerPago', 'fechaUltimoPago', 'fechaCancelacion', 'fechaAprobacion', 'fechaContraActual', 'montoValorFiscal', 'estado', 'idUsuario', 'numReciboCan', 'montoSaldoCan', 'periodoGracia', 'primeraCuota', 'tipoAddendum', 'diasGraciaPrimerPago', 'montoiva', 'mesesGracia', 'anioCancelacion', 'cancelacion', 'skCliente', 'codigoCliente', 'skActivo', 'FechaCreacion', 'EsActual']


In [11]:
cols_fijas = ["numContrato", "idCliente", "idContrato", "skCliente", "skActivo", "fechainicio", "FechaCreacion", "EsActual"]

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 13, Finished, Available, Finished)

In [12]:
metricas = [c for c in df_fact.columns if c not in cols_fijas and not c.startswith("sk") and c not in ["codigoCliente"]]

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 14, Finished, Available, Finished)

In [13]:
cols_silver = cols_fijas + metricas

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 15, Finished, Available, Finished)

In [14]:
df_fact_silver = df_fact.select(*[c for c in cols_silver if c in df_fact.columns])

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 16, Finished, Available, Finished)

In [15]:
display(df_fact_silver)

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 1ba95f63-0dd3-4503-bd3a-6f738aeaa0d7)

In [16]:
df_fact_silver.write.mode("overwrite").format("delta").saveAsTable("lakehouseSilver.factContratosSilver")

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 18, Finished, Available, Finished)

In [17]:
print("✅ factContratosSilver creada correctamente.")

StatementMeta(, e25be9f2-78d0-4b1b-b997-59c93e0e7091, 19, Finished, Available, Finished)

✅ factContratosSilver creada correctamente.
